In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:

# Paths to your and MTAG .best files
your_prs = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/trial04.best'
craig_prs = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/trial05.best'

# Load the .best files
your_prs_df = pd.read_csv(your_prs, sep=r'\s+',header=0, encoding='ascii',engine='python')
craig_prs_df = pd.read_csv(craig_prs, sep=r'\s+',header=0, encoding='ascii',engine='python')

# Display the first few rows to ensure they've loaded correctly
print(your_prs_df.head())
print(craig_prs_df.head())


In [ ]:
# Filter dataframes to include only rows where In_regression is 'yes'
your_prs_filtered = your_prs_df[your_prs_df['In_Regression'] == 'Yes']
craig_prs_filtered = craig_prs_df[craig_prs_df['In_Regression'] == 'Yes']


In [ ]:
# Ensure the dataframes are aligned on the same samples
merged_df = pd.merge(your_prs_filtered, craig_prs_filtered, on='FID', suffixes=('_your', '_craig'))


In [ ]:
# Standardize the PRS values in each dataset separately
your_prs_mean = merged_df['PRS_your'].mean()
your_prs_std = merged_df['PRS_your'].std()
craig_prs_mean = merged_df['PRS_craig'].mean()
craig_prs_std = merged_df['PRS_craig'].std()

merged_df['PRS_your_standardized'] = (merged_df['PRS_your'] - your_prs_mean) / your_prs_std
merged_df['PRS_craig_standardized'] = (merged_df['PRS_craig'] - craig_prs_mean) / craig_prs_std


In [ ]:
# Extract standardized PRS columns for comparison
your_prs_standardized = merged_df['PRS_your_standardized']
craig_prs_standardized = merged_df['PRS_craig_standardized']



In [ ]:
# Calculate the mean and difference of the standardized PRS
mean_prs = (your_prs_standardized + craig_prs_standardized) / 2
diff_prs = your_prs_standardized - craig_prs_standardized


In [ ]:
# Calculate mean difference and limits of agreement
mean_diff = diff_prs.mean()
sd_diff = diff_prs.std()
loa_upper = mean_diff + 1.96 * sd_diff
loa_lower = mean_diff - 1.96 * sd_diff



In [ ]:
# Create the Bland-Altman plot with standardized PRS
plt.figure(figsize=(8, 6))
plt.scatter(mean_prs, diff_prs, alpha=0.5)
plt.axhline(mean_diff, color='gray', linestyle='--')
plt.axhline(loa_upper, color='red', linestyle='--')
plt.axhline(loa_lower, color='blue', linestyle='--')

# Add text annotations on the right side
plt.text(1.05, mean_diff, f'Mean: {mean_diff:.2f}', color='black', va='center', ha='left', rotation=0)
plt.text(1.05, loa_upper, f'+1.96 SD: {loa_upper:.2f}', color='black', va='center', ha='left', rotation=0)
plt.text(1.05, loa_lower, f'-1.96 SD: {loa_lower:.2f}', color='black', va='center', ha='left', rotation=0)

plt.xlabel('Mean of the PRSs (Standardised)', fontsize =16)
plt.ylabel('Difference between the PRSs (Standardized)',fontsize =16)
plt.title('Bland-Altman Plot (Standardized PRS)')
plt.ylim(diff_prs.min() - 0.5, diff_prs.max() + 0.5)  # Adjust ylim to fit annotations
plt.grid(True)
plt.show()

In [ ]:
# Create the Bland-Altman plot with standardized PRS
plt.figure(figsize=(16, 10))
plt.scatter(mean_prs, diff_prs, alpha=0.5)
plt.axhline(mean_diff, color='gray', linestyle='--')
plt.axhline(loa_upper, color='red', linestyle='--')
plt.axhline(loa_lower, color='blue', linestyle='--')

# Adjust the x-axis limit to leave space for annotations
plt.xlim(mean_prs.min() - 0.5, mean_prs.max() + 1.5)

# Add text annotations on the right side
plt.text(mean_prs.max() + 1.8, mean_diff, f'Mean: {mean_diff:.2f}', color='black', va='center', ha='left', rotation=0)
plt.text(mean_prs.max() + 1.8, loa_upper, f'+1.96 SD: {loa_upper:.2f}', color='black', va='center', ha='left', rotation=0)
plt.text(mean_prs.max() + 1.8, loa_lower, f'-1.96 SD: {loa_lower:.2f}', color='black', va='center', ha='left', rotation=0)

plt.xlabel('Mean of the PRSs (Standardised)', fontsize=14)
plt.ylabel('Difference between the PRSs (Standardized)', fontsize=14)
plt.ylim(diff_prs.min() - 0.5, diff_prs.max() + 0.5)  # Adjust ylim to fit annotations
plt.grid(True)

# Display the combined figure
plt.tight_layout()

# Ensure the folder exists and save the figure
folder_path = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Graph_Images/facet_plots/'
file_path = os.path.join(folder_path, 'Bland_altman.pdf')
os.makedirs(folder_path, exist_ok=True)
plt.savefig(file_path)

# Close the plot
plt.close()